# SLB Test Tool

## Setup

### Install Selenium

In [13]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output(wait=True)

%pip install datatable
clear_output()

print("All packages installed")

All packages installed


In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr

import ipywidgets as widgets
import datatable as dt
from datatable import Frame

import pandas as pd
from IPython.display import display
import time


prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()

clear_output()
print("Done!")

Done!


## Source Code

### Generate Objects Keys

In [9]:
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts))
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"

print("Done!")

Done!


### Generate Objects Selectors

In [10]:
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None

print("Done!")

Done!


### GUI

In [30]:
global test_window
test_window = None


def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver


def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result


def start_playground():
    df = pd.DataFrame(columns=["Alias", "Key", "Selector", "Tag Name", "Visibility", "Select"])
    df_original = pd.DataFrame()
    selected_rows = []


    def on_new_window_clicked(b=None):
        global test_window
        try: 
            test_window.quit()
        except: 
            pass  # ignore exceptions
        test_window = new_window("https://life.stg.wellzesta.com/login")


    def on_metadata_clicked(b):
        clear_output(wait=True)
        global test_window, df
        if test_window is None:
            on_new_window_clicked()

        metadata = read_page_objects_metadata(test_window)
        rows = []
        for tag_name, elements in metadata.items():
            for element in elements:
                rows.append({
                    "Tag Name": tag_name,
                    "Alias": element.get('key', ''),
                    "Selector": element.get('selector', ''),
                    "Visibility": element.get('visibility', ''),
                    "Attributes": str(element.get('attributes', '')),  # Convertendo atributos em string para exibição
                    "Text": element.get('text', ''),
                    "Select": False  # Inicializando a coluna Select como False
                })

        df = pd.DataFrame(rows)
        update_table()
        print(f"Page Objects Found: {len(metadata.items())}")


    def on_select_row(change, index):
        print(change.new)
        #df.loc[index, 'Select'] = change.new


    def on_clear_selection_clicked(b):
        global df
        df['Select'] = False
        update_table()


    def on_filter_selected_clicked(b):
        global df
        df = df[df['Select'] == True]
        update_table()


    def on_filter_text_changed(change):
        global df
        filter_text = change.new.lower()
        if filter_text:
            df = df_original[df_original.apply(lambda row: filter_text in str(row).lower(), axis=1)]
        else:
            df = df_original
        update_table()


    def update_table():
        global df

        clear_output(wait=True)
        display(widgets.HBox([new_window_button, metadata_button, filter_text, clear_selection_button, filter_selected_button]))
        
        # Cria um GridspecLayout para a exibição dos dados
        grid = widgets.GridspecLayout(len(df) + 1, len(df.columns) + 1)  # +1 para a coluna de checkboxes

        # Cabeçalhos
        for i, col in enumerate(df.columns):
            grid[0, i] = widgets.Label(value=col)
        grid[0, len(df.columns)] = widgets.Label(value='Select')

        # Dados e checkboxes
        for i, (index, row) in enumerate(df.iterrows()):
            for j, col in enumerate(df.columns):
                if col == 'Select':  # Cria um checkbox para a coluna 'Select'
                    checkbox = widgets.Checkbox(value=row[col])
                    checkbox.observe(lambda change, index=index: on_select_row(change, index), names='value') 
                    grid[i + 1, j] = checkbox
                else:
                    value = row[col]
                    grid[i + 1, j] = widgets.Label(value=str(value))

        display(grid)




    # Widgets
    new_window_button = widgets.Button(description="New Window")
    metadata_button = widgets.Button(description="Inspect")
    clear_selection_button = widgets.Button(description="Clear Selection")
    filter_selected_button = widgets.Button(description="Filter Selected")
    filter_text = widgets.Text(placeholder='Type to filter...')
    

    # Eventos
    new_window_button.on_click(on_new_window_clicked)
    metadata_button.on_click(on_metadata_clicked)
    clear_selection_button.on_click(on_clear_selection_clicked)
    filter_selected_button.on_click(on_filter_selected_clicked)
    filter_text.observe(on_filter_text_changed, names='value')


    # Inicializa a interface sem dados
    update_table()
    

print("Done!")

start_playground()

GridspecLayout(children=(Label(value='Tag Name', layout=Layout(grid_area='widget001')), Label(value='Alias', l…

## Playground

In [12]:
start_playground()

,Tag Name,Alias,Selector,Visibility,Attributes,Text
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,button,btn_Back,"//button[contains(text(), ""Back"")]",visible,"{'type': 'button', 'class': ['btn', 'action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0',…",Back
1,button,btn_Sign-in_using_PIN,"//button[contains(text(), ""Sign-in using PIN"")]",visible,"{'type': 'submit', 'class': ['btn', 'action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0',…",Sign-in using PIN
2,input,txt_email,"//input[@id=""email"" and @name=""email"" and @placeholder=""Type your email""]",visible,"{'type': 'email', 'class': ['px-2', 'w-full', 'rounded', 'h-11', 'bg-gray-300', 'font-bold', 'border…",
3,a,lnk_Terms_&_Conditions,NA,visible,"{'class': ['action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0', 'focus-visible:ring-blue…",Terms & Conditions
4,a,lnk_Sign-in_as_Community,"//a[@data-test-id=""login-as-community""]",visible,"{'class': ['action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0', 'focus-visible:ring-blue…",Sign-in as Community
5,a,lnk_using_PIN,"//a[contains(text(), ""using PIN"")]",visible,"{'class': ['action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0', 'focus-visible:ring-blue…",using PIN
